In [3]:
#Borrar entorno
rm(list=ls())

# Librerías necesarias====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(modeest,
  tidyverse, srvyr, showtext,survey)

In [4]:
#CONSIDERACIONES
#Tiempo de espera para la descarga
options(timeout = 8000)
#Fuente Roboto
font_add_google(name = "Roboto", family = "Roboto")
showtext_auto()
#Ajuste para la UPM
options(survey.lonely.psu = "certainty")  

#### Funciones para descargar y leer datos tanto de la Enigh, como del Censo y la Intercensal

In [9]:
#Datos de la ENIGH ====
leer_enigh <- function(anio, tipo = c("gastospersona", "concentradohogar")) {
  tipo <- match.arg(tipo)

  # Definir las URLs según el tipo
  urls <- list(
    "gastospersona" = list(
      "2016" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2016/microdatos/enigh2016_ns_gastospersona_csv.zip",
      "2018" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2018/microdatos/enigh2018_ns_gastospersona_csv.zip",
      "2020" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2020/microdatos/enigh2020_ns_gastospersona_csv.zip",
      "2022" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_gastospersona_csv.zip",
      "2024" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_gastospersona_csv.zip"
    ),
    "concentradohogar" = list(
      "2016" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2016/microdatos/enigh2016_ns_concentradohogar_csv.zip",
      "2018" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2018/microdatos/enigh2018_ns_concentradohogar_csv.zip",
      "2020" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2020/microdatos/enigh2020_ns_concentradohogar_csv.zip",
      "2022" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/microdatos/enigh2022_ns_concentradohogar_csv.zip",
      "2024" = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2024/microdatos/enigh2024_ns_concentradohogar_csv.zip"
    )
  )
  
  # Archivos a buscar según el tipo
  patrones <- list(
    "gastospersona" = "gastospersona.csv",
    "concentradohogar" = "concentradohogar.csv"
  )

  # Crear archivos temporales
  temp_zip <- tempfile(fileext = ".zip")
  temp_dir <- tempdir()
  options(timeout = 800)

  # Descargar y descomprimir
  download.file(urls[[tipo]][[as.character(anio)]], temp_zip, mode = "wb")
  unzip(temp_zip, exdir = temp_dir)

  # Leer el archivo
  archivo_datos <- list.files(temp_dir, pattern = patrones[[tipo]], 
                              full.names = TRUE, recursive = TRUE)

  datos <- readr::read_csv(archivo_datos, col_types = NULL, show_col_types = FALSE)%>%
  mutate(cve_ent = substr(folioviv, 1, 2))


  # Limpiar archivos temporales
  unlink(temp_zip)
  unlink(list.files(temp_dir, full.names = TRUE), recursive = TRUE)

  return(datos)
}


In [10]:
#Datos deL Censo ====
leer_censo <- function(anio_ent) {  
  url <- urls[[anio_ent]]
  nombre_zip <- paste0(anio_ent, ".zip")
  temp_dir <- tempdir()
  zip_path <- file.path(temp_dir, nombre_zip)
  
  options(timeout = 8000)
  download.file(url, destfile = zip_path, mode = "wb")
  
  unzip(zip_path, exdir = temp_dir)
  
  datos_csv <- list.files(temp_dir, pattern = "\\.CSV$", full.names = TRUE, recursive = TRUE)
  archivo_personas <- datos_csv[grepl("PER|PERS|PERSONA|TR_PERSONA", datos_csv, ignore.case = TRUE)]

  if (length(archivo_personas) == 0) {
    warning(glue::glue("No se encontró archivo esperado para el año {anio_ent}. Se usará el primer CSV disponible."))
    archivo_personas <- datos_csv[1]
  } else {
    archivo_personas <- archivo_personas[1]
  }

  
  datos <- readr::read_csv(archivo_personas, col_types = NULL)%>%
  janitor::clean_names() %>%
   select(ent, mun, factor, upm, estrato, 
   asisten, conact, ent_pais_asi, mun_asi, ent_pais_trab, mun_trab, tie_traslado_escu, tie_traslado_trab)
  
  unlink(zip_path)
  unlink(list.files(temp_dir, full.names = TRUE), recursive = TRUE)
  
  return(datos)
}

#### Porcentaje de hogares en donde el 50% o más de sus integrantes usan el transporte público masivo

In [ ]:
years<-c(2016, 2018, 2020, 2022, 2024)
# Leer los datos de la ENIGH para los años especificados. Incorporar el año como una columna
#Hogares
hogares <- bind_rows(lapply(years, function(x) {
  leer_enigh(x,"concentradohogar") %>%
    mutate(year = x)
}))


#Determinar personas que gastan en transporte público masivo. 
#El transporte público masivo como metro o tren ligero, autobús o trolebús o metrobús.

transporte_pub<- bind_rows(lapply(years, function(x) {
  leer_enigh(x,"gastospersona") %>%
  filter(cve_ent == "09")%>%
    mutate(year = x,
    clasif_gasto = case_when(clave %in% c("B001","B002","B003") ~ "transporte publico masivo",
                                clave %in% c("B004", "B005", "B006", "B007") ~ "otros transportes publicos", 
                                clave %in% c("M001", "M002", "M003", "M004", "M005", "M006") ~ "transporte foraneo",
                                clave %in% c("M007", "M008", "M009", "M010", "M011") ~ "adquisición de vehículos", 
                                clave %in% c("M012", "M013", "M014", "M015", "M016", "M017", "M018") ~ "Refacciones para vehiculos", 
                                clave %in% c("F007", "F008", "F014") ~ "Combustibles para vehículos",
                                clave %in% c("F001", "F002", "F003", "F004", "F005", "F006","R005", "R006", "R007", "R008", "R010", "R011") 
                                ~ "comunicaciones", 
                                TRUE ~ "OTROS GASTOS"),
         transporte = case_when(clasif_gasto == "transporte publico masivo" ~ 1, 
                                clasif_gasto %in% c("transporte foraneo","otros transportes publicos","adquisición de vehículos",
                                                  "Refacciones para vehiculos","Combustibles para vehículos",
                                                  "comunicaciones") ~ 0))%>%
                                                  select(folioviv, foliohog,numren,transporte,year)%>%
                                            filter(transporte==1)%>%
                                           distinct(folioviv, foliohog, numren,year, .keep_all = FALSE)%>%
                                           mutate(transporte=1)%>%
                                           group_by(folioviv,foliohog,year)%>%
                                           summarise(transp_pub_masivo=sum(transporte))
}))

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
`summarise()` has grouped output by 'folioviv', 'foliohog'. You can override
using the `.groups` argument.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
`summarise()` has grouped output by 'folioviv', 'foliohog'. You can override
using the `.groups` argument.
`summarise()` has grouped output by 'folioviv', 'foliohog'. You can override
using the `.groups` argument.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
`summarise()` has grouped output by 'folioviv', 'foliohog'. You can override
using the `.groups` argument.
Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
`summarise()` ha

In [17]:
hog_transporte<-list()

for (anio in years) {
datos_anio <- hogares %>% filter(year == anio)%>%
  left_join(transporte_pub, by = c("folioviv", "foliohog","year")) %>%
  #Calcular porcentaje de residentes que usan transporte público
  mutate(porcentaje_transporte_pub = transp_pub_masivo / tot_integ*100,
         mayoria=case_when(porcentaje_transporte_pub >= 50 ~ 1, 
                                            TRUE ~ 0))

dm<- datos_anio %>%
    as_survey_design(ids = upm, 
                     weights = factor, 
                     nest = TRUE)

dm%>%
filter(
  cve_ent == "09") %>%
group_by(mayoria)%>%
summarise(hogares=survey_total(vartype="cv"),
porcentaje=survey_prop(vartype="cv")*100)%>%
mutate(año = anio) -> resultado
# Almacenar resultados en la lista
  hog_transporte[[as.character(anio)]] <- resultado



}

# Unir resultados en un solo data frame
uso_tp <- bind_rows(hog_transporte, .id = "year")%>%
mutate(tipo="Observado")#%>%
#filter(mayoria==1)
uso_tp

Warning message:
"There was 1 warning in `dplyr::summarise()`.
ℹ In argument: `porcentaje = survey_prop(vartype = "cv") * 100`.
ℹ In group 1: `mayoria = 0`.
Caused by warning:
! glm.fit: algorithm did not converge"


year,mayoria,hogares,hogares_cv,porcentaje,porcentaje_cv,año,tipo
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2016,0,1962181,0.05935186,69.45528,2.211090,2016,Observado
2016,1,862919,0.07562500,30.54472,5.027771,2016,Observado
2018,0,1821887,0.06020160,65.56281,2.233444,2018,Observado
2018,1,956955,0.07035001,34.43719,4.252115,2018,Observado
2020,0,2234681,0.05260629,81.80601,1.330222,2020,Observado
2020,1,497002,0.07792825,18.19399,5.981106,2020,Observado
2022,0,2174089,0.05448266,72.71128,1.774589,2022,Observado
2022,1,815941,0.06881882,27.28872,4.728422,2022,Observado
2024,0,3082330,0.05108574,100.00000,0.000000,2024,Observado


#### Porcentaje de hogares que destinan más del 10% de su ingreso a transporte

In [13]:
# Crear lista para almacenar resultados
resultados <- list()

# Iterar sobre cada año
for (anio in years) {
  
  # Filtrar 
  datos_anio <- hogares %>% filter(year == anio)
  # Crear objeto de diseño muestral
  dm<-datos_anio %>%
    as_survey_design(ids = upm, 
                     weights = factor, 
                     nest = TRUE,
                     strata= est_dis)

  # Calcular indicador
dm%>%
filter(
  cve_ent == "09") %>% # Filtrar por la Ciudad de México
mutate(decil_ing=ntile(ing_cor, 10)) %>%
 filter(ing_cor > 0,
         decil_ing <= 4)%>%
  mutate(porc_ingresos = (publico * 100) / ing_cor,
         prc_ing10 = case_when(porc_ingresos >= 10 ~ "El 10 o más", TRUE ~ "Menos del 10"))%>%
         group_by(prc_ing10) %>%
         summarise(hogares= survey_total(vartype = "cv"),
                    porcentaje = survey_prop(vartype = "cv") * 100) %>%
                    filter(prc_ing10 == "El 10 o más") %>%
          mutate(año = anio) -> resultado
  # Almacenar resultados en la lista
  resultados[[as.character(anio)]] <- resultado
}

# Unir resultados en un solo data frame
prc_gasto_tp <- bind_rows(resultados, .id = "year")%>%
mutate(tipo="Observado")
prc_gasto_tp

year,prc_ing10,hogares,hogares_cv,porcentaje,porcentaje_cv,año,tipo
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2016,El 10 o más,227360,0.08415908,21.51820,7.747506,2016,Observado
2018,El 10 o más,236936,0.08708021,24.90697,7.801374,2018,Observado
2020,El 10 o más,132530,0.11054625,14.28969,9.748595,2020,Observado
2022,El 10 o más,168530,0.09526253,17.02117,8.750173,2022,Observado
2024,El 10 o más,115435,0.12418239,10.72446,12.091749,2024,Observado


#### Porcentaje de población cuyo tiempo de traslado al trabajo es mayor a 60 minutos

In [ ]:
  # Lista de URLs por clave compuesta "anio_ent"
  urls <- list(
    "2015" = "https://www.inegi.org.mx/contenidos/programas/intercensal/2015/microdatos/eic2015_09_csv.zip",
    "2020" = "https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_cdmx_csv.zip"
  )

In [ ]:
#Leer datos de censo e intercensal

datos <- imap_dfr(urls, ~ leer_censo(.y) %>% mutate(anio = .y))

Warning message:
"One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)"
Rows: 553032 Columns: 86
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (20): ENT, NOM_ENT, MUN, NOM_MUN, LOC50K, NOM_LOC, ESTRATO, UPM, ENT_PAI...
dbl (65): ID_VIV, ID_PERSONA, COBERTURA, FACTOR, NUMPER, SEXO, EDAD, PARENT,...
lgl  (1): EDAD_MORIR_D

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 276007 Columns: 91
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (34): ENT, MUN, LOC50K, ID_VIV, ID_PERSONA, ESTRATO, UPM, CLAVIVP, NUMPE...
dbl (57): COBERTURA, FACTOR, SEXO, EDAD, PARENTESCO, NACIONALIDAD, AFRODES, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_type

In [ ]:
datos%>%
group_by(anio)%>%
summarise(registros=n())

anio,registros
<chr>,<int>
2015,553032
2020,276007


In [ ]:
##Calcular tiempos de trabajo al trabajo
tiempos<-list()


for (anio in c("2015", "2020")) {
  datos_anio <- datos %>% filter(anio == !!anio)
  
  dm <- datos_anio %>%
    as_survey_design(ids = upm, 
                     weights = factor, 
                     nest = TRUE,
                     strata= estrato)
  
  
  dm %>%
    filter(ent_pais_trab == "009") %>% # Filtrar solo los que trabajan en CDMX
    mutate(tiempo_trabajo=
    case_when(tie_traslado_trab %in% c(4,5) ~ "Más de una hora",
    TRUE ~ "Menos de una hora, no se traslada o no especificado"))%>%
    group_by(tiempo_trabajo) %>%
    summarise(personas = survey_total(vartype = "cv"),
              porcentaje = survey_prop(vartype = "cv") * 100) %>%
              ungroup() %>%
    mutate(año = anio)%>%
    filter(tiempo_trabajo == "Más de una hora")-> resultado
  
  # Almacenar resultados en la lista
  tiempos[[as.character(anio)]] <- resultado
}

# Unir resultados en un solo data frame
tiempos_trabajo <- bind_rows(tiempos, .id = "anio") %>%
  mutate(tipo = "Observado")
tiempos_trabajo

anio,tiempo_trabajo,personas,personas_cv,porcentaje,porcentaje_cv,año,tipo
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
2015,Más de una hora,662806,0.01925339,17.54626,1.306145,2015,Observado
2020,Más de una hora,744508,0.02986547,17.80950,2.058150,2020,Observado


### Proyecciones